<a href="https://colab.research.google.com/github/ryannemilligan/Parkinsons-Lifestyle-Analysis/blob/main/Parkinsons_Lifestyle_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parkinson’s Disease Risk Factors Explorer

In this project, I will explore how lifestyle, clinical, and cognitive variables relate to Parkinson’s Disease diagnosis. This project is especially meaningful to me, as my grandfather, Fred O’Brien, battled Parkinson’s. His journey has fueled my passion for using data and AI to better understand this disease and improve patient outcomes.

## 1. Load Libraries and Dataset

## 2. Overview and Preprocessing

## 3. Exploratory Data Analysis (EDA)

## 4. Statistical Tests

## 5. Predictive Modeling

## 6. Insights & Export for Tableau